<h1>Data Section - Battle of Neighborhood</h1>

In [3]:
#import packages
import numpy as np # library to handle data in a vectorized manner
from bs4 import BeautifulSoup
import json # library to handle JSON files
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

# Matplotlib and associated plotting modules
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


<h3>scrapping data from wikipedia to derive the Malaysia postcode by states data</h3>

In [4]:
#scrapping using BeautifulSoup
source = requests.get('https://ms.wikipedia.org/wiki/Senarai_poskod_di_Malaysia#Kuala_Lumpur').text
soup = BeautifulSoup(source, 'lxml')
#article = soup.find('a', href='/wiki/States_of_Malaysia', title='States of Malaysia')
#article = soup.find_all('table', class_='wikitable')
article = soup.find_all('table', class_='wikitable')

#output the data into 2 separate lists    
postcode_list=[]
state_list=[]
i=1
#for tag in soup.find_all('td'):
for tag in soup.find_all('td'):
    if i == 1:
        posttemp = tag.text
        posttemp = posttemp.replace('\n','')
        postcode_list.append(posttemp)
    if i == 2:
        statetemp = tag.text
        statetemp = statetemp.replace('\n','')
        state_list.append(statetemp)
    i = i+1
    if i==3:
        i=1

<h3>create a dataframe to keep the postcode data for Kuala Lumpur</h3>

In [5]:
# Creating a dataframe object from listoftuples
mydict = {'PostalCode': postcode_list, 'State': state_list}
dfObjKL = pd.DataFrame({ key:pd.Series(value) for key, value in mydict.items() })

#Only process the cells that have State = 'Kuala Lumpur'
indexNames = dfObjKL[ dfObjKL['State'] != 'Kuala Lumpur' ].index
 
# Delete these row indexes from dataFrame
dfObjKL.drop(indexNames , inplace=True)

#reset index start from 0
dfObjKL = dfObjKL.reset_index(drop=True)

#convert PostalCode to integer for later use
dfObjKL['PostalCode']=dfObjKL['PostalCode'].astype(int)

#dict_df
dfObjKL

,PostalCode,State
0,50000,Kuala Lumpur
1,50050,Kuala Lumpur
2,50100,Kuala Lumpur
3,50150,Kuala Lumpur
4,50200,Kuala Lumpur
5,50250,Kuala Lumpur
6,50300,Kuala Lumpur
7,50350,Kuala Lumpur
8,50400,Kuala Lumpur
9,50450,Kuala Lumpur


<h3>create a dataframe to keep the postcode data for Pulau Pinang</h3>

<h5>Note: Pulau Pinang = Penang</h5>

In [6]:
# Creating a dataframe object from listoftuples
mydict = {'PostalCode': postcode_list, 'State': state_list}
dfObjPenang = pd.DataFrame({ key:pd.Series(value) for key, value in mydict.items() })

#Only process the cells that have State = 'Pulau Pinang'
indexNames = dfObjPenang[ dfObjPenang['State'] != 'Pulau Pinang' ].index
 
# Delete these row indexes from dataFrame
dfObjPenang.drop(indexNames , inplace=True)

#reset index start from 0
dfObjPenang = dfObjPenang.reset_index(drop=True)

#convert PostalCode to integer for later use
dfObjPenang['PostalCode']=dfObjPenang['PostalCode'].astype(int)

dfObjPenang

,PostalCode,State
0,10000,Pulau Pinang
1,10050,Pulau Pinang
2,10100,Pulau Pinang
3,10150,Pulau Pinang
4,10200,Pulau Pinang
5,10250,Pulau Pinang
6,10300,Pulau Pinang
7,10350,Pulau Pinang
8,10400,Pulau Pinang
9,10450,Pulau Pinang


<h3>read the Malaysia polulation data from csv into dataframe</h3>

In [7]:
#read the Malaysia polulation data from csv into dataframe
#Source: National Census 2010,[1] Department of Statistics Malaysia
dfPopulation = pd.read_csv("Malaysia_population_2010.csv", encoding='windows-1252') 

# Preview the first 16 lines of the loaded data 
dfPopulation.head(16)

,State,Population,Area (km2),Pop density,Urban pop (%),Bumiputra (%),Chinese (%),Indian (%)
0,Johor,"3,348,283","19,210",174,71.9,58.9,33.6,7.1
1,Kedah,"1,890,098","9,500",199,64.6,77.9,13.6,7.3
2,Kelantan,"1,459,994","15,099",97,42.4,95.7,3.4,0.3
3,Malacca,"788,706","1,664",470,86.5,66.9,26.4,6.2
4,Negeri Sembilan,"997,071","6,686",150,66.5,61.3,23.2,15.2
5,Pahang,"1,443,365","36,137",40,50.5,79.0,16.2,4.4
6,Penang,"1,520,143","1,048","1,500",90.8,43.6,45.6,10.4
7,Perak,"2,258,428","21,035",110,69.7,57.0,30.4,12.2
8,Perlis,"227,025",821,280,51.4,88.4,8.0,1.2
9,Selangor,"5,411,324","8,104",670,91.4,57.1,28.6,13.5
